In [23]:
from datetime import datetime, timedelta
import yfinance as yf
from langchain_core.tools import tool
from typing import List
from pydantic import BaseModel, Field
import pandas as pd 
import openai
import os
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from operator import itemgetter
from typing import Dict, List, Union

from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType



In [ ]:
os.environ["OPENAI_API_TYPE"]       = ""
os.environ["OPENAI_API_BASE"]       = ""
os.environ["OPENAI_API_KEY"]        = ""
os.environ["OPENAI_API_VERSION"]    = ""
openai.api_type     = os.environ["OPENAI_API_TYPE"]
openai.api_base     = os.environ["OPENAI_API_BASE"]
openai.api_key      = os.environ["OPENAI_API_KEY"] 
openai.api_version  = os.environ["OPENAI_API_VERSION"]


In [ ]:
llm = AzureChatOpenAI(deployment_name='' , verbose=False , temperature=0)

In [73]:
################ This is a function we defined to calculate best performing stocks from Yfinance API

def calculate_performance(symbol, days_ago):
    ticker = yf.Ticker(symbol)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_ago)
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    historical_data = ticker.history(start=start_date, end=end_date)
    old_price = historical_data['Close'].iloc[0]
    new_price = historical_data['Close'].iloc[-1]
    percent_change = ((new_price - old_price) / old_price) * 100
    return round(percent_change, 2)

########## Here we are defining tool and adding the decorator tool from lagchain.

@tool
def get_best_performing(stocks, days_ago):
    """
    Get the best performing stock from a list of stocks.

    Args:
    - stocks (str): A comma-separated string of stock names.
    - days_ago (int): Number of days ago to consider for performance.

    Returns:
    - Tuple[str, float]: A tuple containing the name of the best performing stock and its performance.
    """
    best_stock = None
    best_performance = None
    for stock in stocks:
        try:
            performance = calculate_performance(stock, days_ago)
            if best_performance is None or performance > best_performance:
                best_stock = stock
                best_performance = performance
        except Exception as e:
            print(f"Could not calculate performance for {stock}: {e}")
    return best_stock, best_performance




In [74]:

######## This is the secound tool we are using which is a sample function to add 2 numbers.
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int

In [76]:
############ We are adding both tools top stock performance and sum of two numbers, and pass it to
##### llm to choose which need to be choosen.
tools = [get_best_performing, add]
llm_with_tools = llm.bind_functions(tools)
tool_map = {tool.name: tool for tool in tools}


In [77]:
######### Ininitalizing agents for the function call to select which tool to use.
open_ai_agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.OPENAI_FUNCTIONS,
                        verbose=True)

In [78]:
######## In this it is calling add functions 
open_ai_agent.run("sum of 244 and 455")



> Entering new AgentExecutor chain...

Invoking: `add` with `{'first_int': 244, 'second_int': 455}`


699The sum of 244 and 455 is 699.

> Finished chain.


'The sum of 244 and 455 is 699.'

In [79]:
########### In this it is calling get_best_performang stocks functions.

open_ai_agent.run("Has google's stock gone up over the past 90 days?")



> Entering new AgentExecutor chain...

Invoking: `get_best_performing` with `{'stocks': 'GOOGL', 'days_ago': 90}`


('L', 3.61)Yes, Google's stock (GOOGL) has gone up over the past 90 days. It has increased by 3.61%.

> Finished chain.


"Yes, Google's stock (GOOGL) has gone up over the past 90 days. It has increased by 3.61%."

In [72]:
open_ai_agent.run("How much has google's stock gone up over the past 3 months?")



> Entering new AgentExecutor chain...

Invoking: `get_best_performing` with `{'stocks': 'google', 'days_ago': 90}`


('e', 4.92)Google's stock has gone up by 4.92% over the past 3 months.

> Finished chain.


"Google's stock has gone up by 4.92% over the past 3 months."